# **Run from Google using Colab**

### Installing YoloV3-TF2

The following code is taken from the module, it installs YoLoV3-TF2 if not already installed.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"

Mounted at /content/drive
Note: using Google CoLab


In [2]:
import sys

!{sys.executable} -m pip install git+https://github.com/zzh8829/yolov3-tf2.git@master

  Cloning https://github.com/zzh8829/yolov3-tf2.git (to revision master) to /tmp/pip-req-build-kfa7wkh3
  Running command git clone -q https://github.com/zzh8829/yolov3-tf2.git /tmp/pip-req-build-kfa7wkh3
  Created wheel for yolov3-tf2: filename=yolov3_tf2-0.1-cp36-none-any.whl size=9219 sha256=ddcdd4149e6666025f124c7a3b942699a076e6d8e38c3f0d011f51a77136c08d
  Stored in directory: /tmp/pip-ephem-wheel-cache-8oypff9t/wheels/59/1b/97/905ab51e9c0330efe8c3c518aff17de4ee91100412cd6dd553
Successfully built yolov3-tf2


The following code is taken from the module, it downloads needed files for YoLoV3-TF2.

In [3]:
import os
if COLAB:
  ROOT = '/content/drive/My Drive/Colab Notebooks/yolo'
else:
  ROOT = os.path.join(os.getcwd(),'data')

ROOT

'/content/drive/My Drive/Colab Notebooks/yolo'

In [0]:
import tensorflow as tf
import os

if COLAB:
  ROOT = '/content/drive/My Drive/Colab Notebooks/yolo'
else:
  ROOT = os.path.join(os.getcwd(),'data')

filename_darknet_weights = tf.keras.utils.get_file(
    os.path.join(ROOT,'yolov3.weights'),
    origin='https://pjreddie.com/media/files/yolov3.weights')
TINY = False

filename_convert_script = tf.keras.utils.get_file(
    os.path.join(os.getcwd(),'convert.py'),
    origin='https://raw.githubusercontent.com/zzh8829/yolov3-tf2/master/convert.py')

filename_classes = tf.keras.utils.get_file(
    os.path.join(ROOT,'coco.names'),
    origin='https://raw.githubusercontent.com/zzh8829/yolov3-tf2/master/data/coco.names')
filename_converted_weights = os.path.join(ROOT,'yolov3.tf')

In [5]:
import sys
!{sys.executable} "{filename_convert_script}" --weights "{filename_darknet_weights}" --output "{filename_converted_weights}"

2020-04-10 05:34:23.207372: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Model: "yolov3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, None,  0                                            
__________________________________________________________________________________________________
yolo_darknet (Model)            ((None, None, None,  40620640    input[0][0]                      
__________________________________________________________________________________________________
yolo_conv_0 (Model)             (None, None, None, 5 11024384    yolo_darknet[1][2]               
_________________________________________________________________________

# Now we have used Colab's GPU to transfer weights, we can take the files to local PC (as cloud server has no concept of "desktop"). Below is my folder structure

Under yolo-slave (folder)
*   yolo slave.ipynb
*   convert.py
*   data (folder)

Under data (folder)
*   coco.names
*   yolov3.weights
*   yolov3.tf.index
*   yolov3.tf.data-00000-of-00002
*   yolov3.tf.data-00001-of-00002

# Starter Code

In [0]:
import time
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (YoloV3, YoloV3Tiny)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs
import sys
from PIL import Image, ImageFile
import requests

# Flags are used to define several options for YOLO.
flags.DEFINE_string('classes', filename_classes, 'path to classes file')
flags.DEFINE_string('weights', filename_converted_weights, 'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')
FLAGS([sys.argv[0]])

# Locate devices to run YOLO on (e.g. GPU)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [7]:
# This assignment does not use the "Tiny version"
if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
else:
    yolo = YoloV3(classes=FLAGS.num_classes)

# Load weights and classes
yolo.load_weights(FLAGS.weights).expect_partial()
print('weights loaded')

class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
print('classes loaded')

weights loaded
classes loaded


In [0]:
import numpy as np
import cv2
from PIL import ImageGrab
import pyautogui
import time

In [0]:
while True:
    img_np=np.array(ImageGrab.grab(bbox=(0,0,1920,1080)))
    img_np=cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    img_raw=tf.convert_to_tensor(img_np, np.uint8)

#     img_raw = tf.image.decode_image(img_tensor, channels=3)
    
    # Preprocess image
    img = tf.expand_dims(img_raw, 0)
    img = transform_images(img, FLAGS.size)
    
    # Desired threshold (any sub-image below this confidence level will be ignored.)
    FLAGS.yolo_score_threshold = 0.2

    # Recognize and report results
    boxes, scores, classes, nums = yolo(img)

    # Display image using YOLO library's built in function
    img = img_raw.numpy()
    img = draw_outputs(img, (boxes, scores, classes, nums), class_names)
    #cv2.imwrite(FLAGS.output, img) # Save the image
#     display(Image.fromarray(img, 'RGB')) # Display the image

    items = []
    # print('detections:')
    for j in range(nums[0]):
        cls = class_names[int(classes[0][j])]
        score = np.array(scores[0][j])
        box = np.array(boxes[0][j])
        items.append(cls)
    
    cv2.imshow('img1_bg',img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()